## Step 1 News Data Cleaning

In [1]:
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
news_df = pd.read_csv('/Users/qlin/Desktop/sp500_focus_sources_1_23_dedup.csv')

In [6]:
# Count the number of unique 'content' values among the entire dataset
unique_entries = news_df['summary'].nunique()
print(f'There are {unique_entries} unique entries in the "summary" column.')

There are 64492 unique entries in the "summary" column.


In [5]:
# Drop the duplicated rows but keep the first occurence of each unique 'summary' value based on 'addDate' column
# First, sort the DataFrame by 'addDate'
df_sorted = news_df.sort_values('addDate')

# Then, drop duplicates based on 'content', keeping the first occurrence
df_dedup = df_sorted.drop_duplicates(subset='summary', keep='first')

# Check the number of rows in the deduplicated DataFrame
print(f'The deduplicated DataFrame has {df_dedup.shape[0]} rows.')

The deduplicated DataFrame has 64493 rows.


## Step 2 Filtering SP100 News Only

In [8]:
import pandas as pd
import ast

# Assume df_dedup is your DataFrame containing deduplicated news articles

# Step 1: Load the top 100 companies' tickers
with open('Data/Top_100_Companies_Tickers.txt', 'r') as file:
    top_100_tickers = file.read().splitlines()

# Step 2: Modify the function
def contains_top_100_company(companies_str):
    companies = ast.literal_eval(companies_str)  # Convert the string to a list
    for company in companies:
        if any(ticker in company['symbols'] for ticker in top_100_tickers):
            return True  # Return True if any of the top 100 companies' tickers is found
    return False  # Return False otherwise

In [9]:
# Select rows where 'symbols' contains any of the top 100 companies' tickers
df_top_100 = df_dedup[df_dedup['companies'].apply(contains_top_100_company)]

print(f'There are {len(df_top_100)} news articles related to the top 100 companies.')

There are 33925 news articles related to the top 100 companies.


In [45]:
# Filter only this columns: 'addDate', 'title', 'description', 'content', 'keywords', 'topics', 'entities', 'companies', 'summary'
news_top_100 = df_top_100[['addDate', 'title', 'entities', 'companies', 'summary']]

In [46]:
news_top_100.to_csv('/Users/qlin/Desktop/news_top_100.csv', index=False)

In [48]:
import json
def parse_company(company_str): 
    '''
    function to parse the company string and return a list of companies
    '''
    try:
        # Try to parse the string as JSON
        companies = json.loads(company_str.replace("'", "\""))
        # Extract the 'name' field from each topic
        return [company['name'] for company in companies]
    except json.JSONDecodeError:
        # If the string is not valid JSON, return an empty list
        return []

news_top_100['parse_companies'] = news_top_100['companies'].apply(parse_company)

/var/folders/3q/t2ddpfgd6gs899m17rdgm2vc0000gn/T/ipykernel_20936/1101133659.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_top_100['parse_companies'] = news_top_100['companies'].apply(parse_company)


In [20]:
top_100_company = pd.read_csv("Data/Top_100_Companies.csv")
top_100_company

,Unnamed: 0,Ticker,Description,Sector,Market Capitalization
0,1,AAPL,Apple Inc.,Electronic Technology,2728017215293
1,2,MSFT,Microsoft Corporation,Technology Services,2351371643107
2,3,GOOG,Alphabet Inc.,Technology Services,1611856497958
3,4,GOOGL,Alphabet Inc.,Technology Services,1610343852181
4,5,AMZN,"Amazon.com, Inc.",Retail Trade,1366884251763
...,...,...,...,...,...
95,96,AMT,American Tower Corporation (REIT),Finance,82187878166
96,97,CB,Chubb Limited,Finance,82105914715
97,98,CI,The Cigna Group,Health Services,81693476157
98,99,C,"Citigroup, Inc.",Finance,81360929490


In [21]:
# Create a mapping from company descriptions to tickers
description_to_ticker = pd.Series(top_100_company.Ticker.values, index=top_100_company.Description).to_dict()
description_to_ticker

{'Apple Inc.': 'AAPL',
 'Microsoft Corporation': 'MSFT',
 'Alphabet Inc.': 'GOOGL',
 'Amazon.com, Inc.': 'AMZN',
 'NVIDIA Corporation': 'NVDA',
 'Berkshire Hathaway Inc. New': 'BRK.B',
 'Meta Platforms, Inc.': 'META',
 'Tesla, Inc.': 'TSLA',
 'Eli Lilly and Company': 'LLY',
 'Visa Inc.': 'V',
 'UnitedHealth Group Incorporated': 'UNH',
 'Johnson & Johnson': 'JNJ',
 'Exxon Mobil Corporation': 'XOM',
 'JP Morgan Chase & Co.': 'JPM',
 'Walmart Inc.': 'WMT',
 'Mastercard Incorporated': 'MA',
 'Procter & Gamble Company (The)': 'PG',
 'Broadcom Inc.': 'AVGO',
 'Home Depot, Inc. (The)': 'HD',
 'Oracle Corporation': 'ORCL',
 'Chevron Corporation': 'CVX',
 'Merck & Company, Inc.': 'MRK',
 'AbbVie Inc.': 'ABBV',
 'Coca-Cola Company (The)': 'KO',
 'PepsiCo, Inc.': 'PEP',
 'Costco Wholesale Corporation': 'COST',
 'Adobe Inc.': 'ADBE',
 'Bank of America Corporation': 'BAC',
 'Cisco Systems, Inc.': 'CSCO',
 'Pfizer, Inc.': 'PFE',
 'Thermo Fisher Scientific Inc': 'TMO',
 'McDonald’s Corporation': 'MCD

In [38]:
def calculate_mentions(companies, entities):
    '''
    checks if each entity name from entities is a substring of the company name from parse_companies. 
    If it is, it increments the mention count for that company.
    '''
    mentions = {company: 0 for company in companies}
    for entity in entities:
        if entity['type'] == 'ORG':
            for company in companies:
                if entity['data'] in company:
                    mentions[company] += entity['mentions']
    return mentions

df_top_100['mentions'] = df_top_100.apply(lambda row: calculate_mentions(row['parse_companies'], row['entities']), axis=1)

/var/folders/3q/t2ddpfgd6gs899m17rdgm2vc0000gn/T/ipykernel_20936/1874158048.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_100['mentions'] = df_top_100.apply(lambda row: calculate_mentions(row['parse_companies'], row['entities']), axis=1)


## Below are not yet tested code

In [ ]:
def calculate_percentage(mentions):
    total_mentions = sum(mentions.values())
    return {company: (count / total_mentions) * 100 for company, count in mentions.items()}

df_top_100['percentage'] = df_top_100['mentions'].apply(calculate_percentage)

In [ ]:
def find_main_company(percentage):
    return max(percentage, key=percentage.get)

df_top_100['main_company'] = df_top_100['percentage'].apply(find_main_company)
df_top_100